In [1]:
import torch

In [3]:
x=torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
y=x+2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [7]:
print(x.grad_fn)
print(y.grad_fn)

None


In [10]:
z=y*y*3
out=z.mean()
print(z,out)
print(z.grad_fn)
print(out.grad_fn)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


In [13]:
a=torch.randn(2,2)
a=(a*3)/(a-1)
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)

False
True


In [14]:
# Gradient
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [15]:
#另外一个雅可比向量积德例子

In [16]:
x=torch.randn(3,requires_grad=True)
y=x*2
while y.data.norm()<1000:
    y=y*2
print(y)

tensor([ 999.3406, -764.2306,  -84.9517], grad_fn=<MulBackward0>)


In [17]:
v=torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [18]:
print(x.requires_grad)
print((x**2).requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


In [31]:
t=x**2
print((x**2).requires_grad)
x.requires_grad_(False)
with torch.no_grad():
    print((x).requires_grad)

True
False


# Restart the kernel at here!!!!!

In [14]:
import torch
# import torch.nn as nn
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=torch.nn.Conv2d(1,6,5)
        self.conv2=torch.nn.Conv2d(6,16,5)
        
        self.fc1=torch.nn.Linear(16*5*5,120)
        self.fc2=torch.nn.Linear(120,84)
        self.fc3=torch.nn.Linear(84,10)
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
params=list(net.parameters())
print(len(params))

print(params[0].size())


10
torch.Size([6, 1, 5, 5])


In [16]:
input=torch.randn(1,1,32,32)
out=net(input)
print(out)



tensor([[-0.0294,  0.1486,  0.0837,  0.0239, -0.0032, -0.0801,  0.0152, -0.0997,
         -0.0413, -0.1052]], grad_fn=<AddmmBackward>)


In [17]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [19]:
output=net(input)
target=torch.randn(10)
target=target.view(1,-1)
criterion=torch.nn.MSELoss()
loss=criterion(output,target)
print(loss)

tensor(0.6576, grad_fn=<MseLossBackward>)


In [22]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [23]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)


tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0033, -0.0033, -0.0042, -0.0037,  0.0067,  0.0025])


In [24]:
import torch.optim as optim
optimizer=optim.SGD(net.parameters(),lr=0.01)
optimizer.zero_grad()
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step()